In [1]:
import logging

In [2]:
def logger_info(logger):
    return {
        'set level': logging.getLevelName(logger.level),
        'actual level': logging.getLevelName(logger.getEffectiveLevel()),
        'use parent handlers': logger.propagate,
        'own handlers': logger.handlers,
    }


def log(logger, x=1):
    logger.debug('debug')
    logger.info('info')
    logger.warning('warning')
    logger.error('error')
    logger.critical('critical')
    try:
        1 / x
    except ZeroDivisionError:
        logger.exception('exception')

## Info

This notebook must be run from start to end (Kernel -> Restart & Run All), since loggers are singletons.  Any re-runing of cells might give unexpected results (although figuring out what is going on could be a good exercise).

## Root logger

In [3]:
root_logger = logging.getLogger('')  # get it by passing an empty string
logger_info(root_logger)

{'set level': 'WARNING',
 'actual level': 'WARNING',
 'use parent handlers': True,
 'own handlers': []}

In [4]:
id(root_logger) == id(logging.root)

True

In [5]:
# Although its name is 'root', it cannot be fetched by using it. Use '' as above.
root_logger.name, id(root_logger) == id(logging.getLogger(root_logger.name))

('root', False)

In [6]:
# output is still present due to last resort handler which is activated if handlers list is empty
log(root_logger)

warning
error
critical


In [7]:
log(root_logger, x=0)  # exception log includes stack trace

warning
error
critical
exception
Traceback (most recent call last):
  File "<ipython-input-2-edd67f1b7ed6>", line 17, in log
    1 / x
ZeroDivisionError: division by zero


### Basic configuration

Calling `basicConfig()` has effect only when handlers list is empty! In python 3.8, `force` argument was added to override.

In [8]:
logging.basicConfig(
    level='ERROR'
)
logger_info(root_logger)

{'set level': 'ERROR',
 'actual level': 'ERROR',
 'use parent handlers': True,
 'own handlers': [<StreamHandler stderr (NOTSET)>]}

In [9]:
log(root_logger)

ERROR:root:error
CRITICAL:root:critical


In [10]:
# will not change anything
logging.basicConfig(
    level='INFO'
)
logger_info(root_logger)

{'set level': 'ERROR',
 'actual level': 'ERROR',
 'use parent handlers': True,
 'own handlers': [<StreamHandler stderr (NOTSET)>]}

In [11]:
log(root_logger)

ERROR:root:error
CRITICAL:root:critical


In [12]:
root_logger.handlers = []  # allows for changing config, in this case: ERROR -> INFO
logging.basicConfig(
    level='INFO'
)
logger_info(root_logger)

{'set level': 'INFO',
 'actual level': 'INFO',
 'use parent handlers': True,
 'own handlers': [<StreamHandler stderr (NOTSET)>]}

In [13]:
log(root_logger)

INFO:root:info
ERROR:root:error
CRITICAL:root:critical


## Child loggers

When created, the level and handlers of the parent (in this case root) will be used.

In [14]:
child_logger = logging.getLogger('child')
logger_info(child_logger)

{'set level': 'NOTSET',
 'actual level': 'INFO',
 'use parent handlers': True,
 'own handlers': []}

In [15]:
log(child_logger)

INFO:child:info
ERROR:child:error
CRITICAL:child:critical


### Override level

In [16]:
child_logger.setLevel('WARNING')
logger_info(child_logger)

{'set level': 'WARNING',
 'actual level': 'WARNING',
 'use parent handlers': True,
 'own handlers': []}

In [17]:
log(child_logger)

ERROR:child:error
CRITICAL:child:critical


### Prevent calling parent handlers

In [18]:
child_logger.propagate = False
logger_info(child_logger)

{'set level': 'WARNING',
 'actual level': 'WARNING',
 'use parent handlers': False,
 'own handlers': []}

In [19]:
log(child_logger)  # since handlers list is empty again, last resort handler is activated

warning
error
critical


### Disable logging

Logging is disabled when `propagate = False` and handlers list contains a single `NullHandler`.  

In [20]:
child_logger.addHandler(logging.NullHandler())
logger_info(child_logger)

{'set level': 'WARNING',
 'actual level': 'WARNING',
 'use parent handlers': False,
 'own handlers': [<NullHandler (NOTSET)>]}

In [21]:
log(child_logger)  # now logging is completely disabled for this logger

### Setup custom handler

In [22]:
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

handler = logging.StreamHandler()
handler.setLevel('CRITICAL')  # handler can have its own severity level
handler.setFormatter(formatter)

child_logger.addHandler(handler)

logger_info(child_logger)

{'set level': 'WARNING',
 'actual level': 'WARNING',
 'use parent handlers': False,
 'own handlers': [<NullHandler (NOTSET)>, <StreamHandler stderr (CRITICAL)>]}

In [23]:
log(child_logger)

2020-04-21 21:18:55,341 - CRITICAL - critical


### Calling same setup again will create duplicate logs

Handlers are appended to a list and each is called in turn.

In [24]:
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

handler = logging.StreamHandler()
handler.setLevel('CRITICAL')  # handler can have its own severity level
handler.setFormatter(formatter)

child_logger.addHandler(handler)

logger_info(child_logger)

{'set level': 'WARNING',
 'actual level': 'WARNING',
 'use parent handlers': False,
 'own handlers': [<NullHandler (NOTSET)>,
  <StreamHandler stderr (CRITICAL)>,
  <StreamHandler stderr (CRITICAL)>]}

In [25]:
log(child_logger)

2020-04-21 21:18:55,376 - CRITICAL - critical
2020-04-21 21:18:55,376 - CRITICAL - critical


### Re-enable parent logging

In [26]:
child_logger.propagate = True
logger_info(child_logger)

{'set level': 'WARNING',
 'actual level': 'WARNING',
 'use parent handlers': True,
 'own handlers': [<NullHandler (NOTSET)>,
  <StreamHandler stderr (CRITICAL)>,
  <StreamHandler stderr (CRITICAL)>]}

In [27]:
log(child_logger)

ERROR:child:error
2020-04-21 21:18:55,422 - CRITICAL - critical
2020-04-21 21:18:55,422 - CRITICAL - critical
CRITICAL:child:critical
